In [ ]:
from carsus.io import GFALLReader
gfall_reader = GFALLReader()

In [ ]:
levels = gfall_reader.extract_levels()
lines = gfall_reader.extract_lines()

In [ ]:
from jinja2 import Template
import pandas as pd
def extract_levels_by_atomic_number(atomic_number, n_rows):
    """
    Extracts the top N rows from the 'Levels' dataframe for a given atomic number.
    """
    df = levels[levels.index.get_level_values('atomic_number') == atomic_number]
    return df.head(n_rows)

def extract_lines_by_atomic_number(atomic_number, n_rows):
    """
    Extracts the top N rows from the 'Lines' dataframe for a given atomic number.
    """
    df = lines[lines.index.get_level_values('atomic_number') == atomic_number]
    return df.head(n_rows)

def generateJinjaTemplate(levels_df, lines_df, filename="output.html"):
    """
    Generates an HTML file displaying a navigation menu to toggle between Levels and Lines tables.
    """
    template = Template("""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <meta name="description" content="HTML report displaying atomic data for Levels and Lines.">
        <title>Silicon Data Viewer</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
            }
            h1 {
                text-align: center;
                color: black;
            }
            .nav-buttons {
                text-align: center;
                margin: 20px 0;
            }
            .nav-buttons button {
                padding: 10px 20px;
                margin: 0 10px;
                font-size: 16px;
                border: none;
                background-color: #007BFF;
                color: white;
                cursor: pointer;
                border-radius: 5px;
                box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);
            }
            .nav-buttons button:hover {
                background-color: #0056b3;
            }
            .table-container {
                display: none;
            }
            .table-container.active {
                display: block;
            }
            table {
                border-collapse: collapse;
                width: 100%;
                margin: 20px 0;
                box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);
                border-radius: 10px;
                overflow: hidden;
            }
            th, td {
                border: 1px solid #ddd;
                padding: 8px;
                text-align: center;
            }
            th {
                background-color: #007BFF;
                color: white;
                font-weight: bold;
            }
            tr:nth-child(even) {
                background-color: #f9f9f9;
            }
            tr:hover {
                background-color: #f1f1f1;
            }
        </style>
        <script>
            function showTable(tableId) {
                // Hides all tables
                document.querySelectorAll('.table-container').forEach(function(table) {
                    table.classList.remove('active');
                });
                // Shows the selected table
                document.getElementById(tableId).classList.add('active');
            }
        </script>
    </head>
    <body>
        <h1>Silicon Data Viewer</h1>
        <div class="nav-buttons">
            <button onclick="showTable('levels-table')">View Levels</button>
            <button onclick="showTable('lines-table')">View Lines</button>
        </div>
        <div id="levels-table" class="table-container active">
            <h2>Display the top 50 rows of the "Levels" dataframe for Silicon.</h2>
            {{ levels_table | safe }}
        </div>
        <div id="lines-table" class="table-container">
            <h2>Display the top 50 rows of the "Lines" dataframe for Silicon.</h2>
            {{ lines_table | safe }}
        </div>
    </body>
    </html>
    """)

    levels_html_table = levels_df.to_html(classes="styled-table", index=True)
    lines_html_table = lines_df.to_html(classes="styled-table", index=True)

    rendered_html = template.render(
        levels_table=levels_html_table,
        lines_table=lines_html_table,
    )

    with open(filename, "w") as file:
        file.write(rendered_html)

    print(f"HTML file '{filename}' generated successfully. Open it to view the output.")

if __name__ == "__main__":

    atomic_number = 14
    top_rows = 50

    try:
        levels_df = extract_levels_by_atomic_number(atomic_number, top_rows)
        lines_df = extract_lines_by_atomic_number(atomic_number, top_rows)
        generateJinjaTemplate(levels_df, lines_df, "Jinja2Silicon.html")
    except Exception as e:
        print(f"Error: {e}")


In [ ]:
from flask import Flask, render_template, request, flash
from flask_ngrok import run_with_ngrok

app = Flask(__name__)

def extract_levels_by_atomic_number(atomic_number, n_rows):
    """
    Extracts the top N rows from the Levels DataFrame for a given atomic number.
    """
    try:
        df = levels[levels.index.get_level_values('atomic_number') == atomic_number]
        return df.head(n_rows)
    except Exception as e:
        raise ValueError(f"Error extracting levels data: {str(e)}")

def extract_lines_by_atomic_number(atomic_number, n_rows):
    """
    Extracts the top N rows from the Lines DataFrame for a given atomic number.
    """
    try:
        df = lines[lines.index.get_level_values('atomic_number') == atomic_number]
        return df.head(n_rows)
    except Exception as e:
        raise ValueError(f"Error extracting lines data: {str(e)}")

@app.route("/", methods=["GET", "POST"])
def index():
    try:
        atomic_number = 14  
        n_rows = 5         
        data_type = "levels"
        if request.method == "POST":
            try:
                atomic_number = int(request.form.get("atomic_number", 14))
                n_rows = int(request.form.get("n_rows", 5))
                data_type = request.form.get("data_type", "levels")
                if atomic_number <= 0 or n_rows <= 0:
                    raise ValueError("Atomic number and rows must be positive integers.")
            except ValueError as ve:
                flash(f"Invalid input: {str(ve)}", "error")
                return render_template("template.html", atomic_number=atomic_number, n_rows=n_rows)

        if data_type == "levels":
            levels_data = extract_levels_by_atomic_number(atomic_number, n_rows)
            table_html = levels_data.to_html(classes='styled-table', index=True)
        else:
            lines_data = extract_lines_by_atomic_number(atomic_number, n_rows)
            table_html = lines_data.to_html(classes='styled-table', index=True)
        return render_template(
            "Jinja2SiliconFlask.html", 
            atomic_number=atomic_number,
            n_rows=n_rows,
            data_type = data_type,
            table_html=table_html
        )

    except ValueError as ve:
        flash(f"Data extraction error: {str(ve)}", "error")
        return render_template("template.html", atomic_number=14, n_rows=5, data_type="levels")

    except Exception as e:
        flash(f"An unexpected error occurred: {str(e)}", "error")
        return render_template("template.html", atomic_number=14, n_rows=5, data_type="levels")

run_with_ngrok(app)
app.run()
